In [1]:
import os
from configs.parser import YAMLParser
import mlflow
from dataloader.h5 import H5Loader
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

from loss.flow import FWL, RSAT, AEE
from utils.utils import load_model
from models.model import (
    FireNet,
    RNNFireNet,
    LeakyFireNet,
    FireFlowNet,
    LeakyFireFlowNet,
    E2VID,
    EVFlowNet,
    RecEVFlowNet,
    LeakyRecEVFlowNet,
    RNNRecEVFlowNet,
)

In [ ]:
%reset -f

In [2]:
runid = "EVFlowNet"

mlflow.set_tracking_uri("")
run = mlflow.get_run(runid)
config_parser = YAMLParser("configs/eval_MVSEC.yml")
config = config_parser.config
config = config_parser.merge_configs(run.data.params)


In [3]:
data = H5Loader(config, config["model"]["num_bins"])
dataloader = torch.utils.data.DataLoader(
        data,
        drop_last=True,
        batch_size=config["loader"]["batch_size"],
        collate_fn=data.custom_collate,
        worker_init_fn=config_parser.worker_init_fn
    )

In [4]:
def visualize_flow(flow):
    hsv = np.zeros((flow.shape[1], flow.shape[2], 3), dtype=np.uint8)
    hsv[..., 1] = 255
    mag, ang = cv2.cartToPolar(flow[0,...].numpy(), flow[1,...].numpy())
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return rgb

In [ ]:
data_number = 0
flow = data[data_number]["gtflow"]


plt.imshow(visualize_flow(flow))
for ev in data[data_number]["event_list"].transpose(1, 0):
    plt.scatter(ev[1], ev[2], s=5, c="r")
plt.show()

In [5]:
class AEE_HIST(AEE):
    
    def __init__(self, config, device, flow_scaling=128):
        super().__init__(config, device, flow_scaling)
        self._masked_error_list = None
        self._masked_gtflow_list = None
    
    def mask_for_flow(self):
        event_mask = self._event_mask[:, -1, :, :].bool()
        gtflow_mask_x = self._gtflow[:, 0, :, :] == 0.0
        gtflow_mask_y = self._gtflow[:, 1, :, :] == 0.0
        gtflow_mask = gtflow_mask_x | gtflow_mask_y
        gtflow_mask = ~gtflow_mask
        mask = event_mask & gtflow_mask
        return mask.squeeze()
    
    def masked_optical_flow(self,flow, mask):
        masked_flow = flow[:,:,mask]
        masked_gtflow = self._gtflow[:,:,mask]
        masked_error = masked_flow - masked_gtflow
        return masked_error.detach().cpu(), masked_gtflow.detach().cpu()
    
    def flow_accumulation(self):
        flow = self._flow_map[-1] * self.flow_scaling
        flow *= self._dt_gt.to(self.device) / self._dt_input.to(self.device)
        mask = self.mask_for_flow()
        masked_error, masked_gtflow = self.masked_optical_flow(flow, mask)
        if self._masked_error_list is None:
            self._masked_error_list = masked_error
        else:
            self._masked_error_list = torch.cat([self._masked_error_list, masked_error], dim=2)
        if self._masked_gtflow_list is None:
            self._masked_gtflow_list = masked_gtflow
        else:
            self._masked_gtflow_list = torch.cat([self._masked_gtflow_list, masked_gtflow], dim=2)
    
    def optical_flow_vec2scalar(self, flow, reduce_norm='L2'):
        if reduce_norm == 'L1':
            return torch.sum(torch.abs(flow), dim=1)
        elif reduce_norm == 'L2':
            return torch.sqrt(torch.sum(flow**2, dim=1))
            
    def calculate_error_hist(self,n_bins=10):
        gtflow_mag = self.optical_flow_vec2scalar(self._masked_gtflow_list).view(-1)
        error_mag = self.optical_flow_vec2scalar(self._masked_error_list).view(-1)
        # Compute histogram and bin centers
        hist, bin_edges = np.histogram(gtflow_mag, bins=n_bins)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

        # Compute mean and standard deviation for each bin
        mean_values = []
        std_values = []
        for i in range(n_bins):
            idx = np.where((gtflow_mag >= bin_edges[i]) & (gtflow_mag < bin_edges[i+1]))
            mean_values.append(np.mean(error_mag[idx]))
            std_values.append(np.std(error_mag[idx]))
            
        # Create plot
        fig, ax = plt.subplots()
        ax.bar(bin_centers, hist, width=bin_centers[1]-bin_centers[0], color='gray', alpha=0.5)
        ax.errorbar(bin_centers, mean_values, yerr=std_values, fmt='o', color='black')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_title('Histogram with Mean and Standard Deviation')
        plt.show()


In [6]:
device = config_parser.device

In [ ]:
model = RecEVFlowNet(config["model"]).to(device)

In [ ]:


model = load_model(runid, model, device)
model.eval()
aee_hist = AEE_HIST(config, device, flow_scaling=config["metrics"]["flow_scaling"])

In [ ]:
end_test = False
c = 0
val_total = 0
print("New sequence: ", data.seq_num)
while True:
    
    for inputs in dataloader:
        
        if data.new_seq:
            data.new_seq = False
            activity_log = None
            model.reset_states()
            print("New sequence: ", data.seq_num)
            print(val_total/c)
            end_test = True
            break
        
        # forward pass
        x = model(
            inputs["event_voxel"].to(device), inputs["event_cnt"].to(device), log=config["vis"]["activity"]
        )
        
        aee_hist.event_flow_association(x["flow"], inputs)   
        val_metric = aee_hist() 
        c += 1
        val_total += val_metric
        aee_hist.flow_accumulation()
        aee_hist.reset()
        # finish inference loop
        if data.seq_num >= len(data.files):
            end_test = True
            break
    
    if end_test:
        break

In [ ]:
n_bins = 20
gtflow_mag = aee_hist.optical_flow_vec2scalar(aee_hist._masked_gtflow_list).view(-1)
error_mag = aee_hist.optical_flow_vec2scalar(aee_hist._masked_error_list).view(-1)
# Compute histogram and bin centers
hist, bin_edges = torch.histogram(gtflow_mag, bins=n_bins)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Compute mean and standard deviation for each bin
mean_values = []
std_values = []
for i in range(n_bins):
    idx = torch.where((gtflow_mag >= bin_edges[i]) & (gtflow_mag < bin_edges[i+1]))
    mean_values.append(torch.mean(error_mag[idx]))
    std_values.append(torch.std(error_mag[idx]))
    
# Create plot
fig, ax1 = plt.subplots()
ax1.bar(bin_centers, hist/gtflow_mag.shape[0], width=bin_centers[1]-bin_centers[0], color='gray', alpha=0.5)
ax1.set_xlabel('X')
ax1.set_ylabel('histogram of GT flow magnitude')
ax2 = ax1.twinx()
ax2.errorbar(bin_centers, mean_values, yerr=std_values, fmt='o', color='black')
ax2.set_ylabel('AEE')
ax.set_title('Histogram with Mean and Standard Deviation')
plt.show()


In [ ]:
val_metric

In [ ]:

# Create some data
x = np.linspace(0, 10, 100)
y1 = np.sin(x)
y2 = np.exp(x)

# Create the first plot
fig, ax1 = plt.subplots()
ax1.plot(x, y1, 'b-')
ax1.set_xlabel('x')
ax1.set_ylabel('sin(x)', color='b')
ax1.tick_params('y', colors='b')

# Create the second plot
ax2 = ax1.twinx()
ax2.plot(x, y2, 'r-')
ax2.set_ylabel('exp(x)', color='r')
ax2.tick_params('y', colors='r')

# Show the plot
plt.show()


In [ ]:
end_test = False
c = 0
print("New sequence: ", data.seq_num)
while True:
    
    for inputs in dataloader:
        c = c + 1
        if data.new_seq:
            data.new_seq = False
            activity_log = None
            model.reset_states()
            print(c)
            c = 0
            print("New sequence: ", data.seq_num)
        
        # forward pass
        x = model(
            inputs["event_voxel"].to(device), inputs["event_cnt"].to(device), log=config["vis"]["activity"]
        )
        
        aee.event_flow_association(x["flow"], inputs)   
        val_metric = aee() 
        aee.reset()
        # finish inference loop
        if data.seq_num >= len(data.files):
            end_test = True
            break
    
    if end_test:
        break


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate sample data
data = np.random.randn(100, 2)  # Replace this with your own data

# Define histogram parameters
n_bins = 10
x_column = 0

# Compute histogram and bin centers
hist, bin_edges = np.histogram(data[:, x_column], bins=n_bins)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Compute mean and standard deviation for each bin
mean_values = []
std_values = []
for i in range(n_bins):
    idx = np.where((data[:, x_column] >= bin_edges[i]) & (data[:, x_column] < bin_edges[i+1]))
    mean_values.append(np.mean(data[idx, 1]))
    std_values.append(np.std(data[idx, 1]))

# Create plot
fig, ax = plt.subplots()
ax.bar(bin_centers, hist, width=bin_centers[1]-bin_centers[0], color='gray', alpha=0.5)
ax.errorbar(bin_centers, mean_values, yerr=std_values, fmt='o', color='black')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Histogram with Mean and Standard Deviation')
plt.show()


In [ ]:
import torch
import numpy as np
import os
import cv2

class DSectDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        
        # Load the list of event and image files
        self.event_files = os.listdir(os.path.join(root_dir, 'events'))
        self.image_files = os.listdir(os.path.join(root_dir, 'images'))
        
        # Sort the files to ensure they match up
        self.event_files.sort()
        self.image_files.sort()

    def __len__(self):
        return len(self.event_files)

    def __getitem__(self, idx):
        # Load the event and image files for the given index
        event_file = os.path.join(self.root_dir, 'events', self.event_files[idx])
        image_file = os.path.join(self.root_dir, 'images', self.image_files[idx])
        
        # Load the event data
        events = np.load(event_file)
        
        # Load the image data
        image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
        
        # Apply transformations to the data if provided
        if self.transform:
            events, image = self.transform((events, image))
        
        # Convert the data to PyTorch tensors
        events = torch.from_numpy(events).float()
        image = torch.from_numpy(image).float()
        
        return events, image


In [ ]:
# Create the dataset
dataset = DSectDataset(root_dir='/path/to/dataset')

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Loop over the dataloader
for events, image in dataloader:
    # Do something with the event and image data
    pass


In [ ]:
class MyClass:
    print("Hello, I am a MyClass!")
    a = 9
    def __init__(self):
        print("Hello, I am an instance of MyClass!")


In [ ]:
sa = MyClass()

sa.a

In [ ]:
np.shares_memory(B, B2)
